## Jonathan Perkins
## 4/18/2024

# Feature generation

We will create machine learning models to predict material properties solely based on their
chemical compositions. The first step is to create input features (or descriptors or attributes).
We will follow the paper by Ward et al. in npj Computational Materials (2016) 2, 16028.
The features are designed to enable machine learning algorithms to construct general rules
that can possibly learn and reflect to some extent physical and chemical intuitions.
Please write a python function with a single string variable of chemical formula (such as
‘Fe2O3’) as the input argument. Upon completion, the function will return a list containing
the following 60 features (floating point or integer values)

# Imports

In [55]:
import pymatgen.core as pmg
from pymatgen.core.periodic_table import Element
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Creating input features

In [57]:
# Valence electrons
val_s = {'Fe':2, 'O':2, 'Sr':2, 'Ti':2}
val_p = {'Fe':0, 'O':4, 'Sr':0, 'Ti':0}
val_d = {'Fe':6, 'O':0, 'Sr':0, 'Ti':2}
val_f = {'Fe':0, 'O':0, 'Sr':0, 'Ti':0}

def find_features(formula):

  # Uses all functions defined below to return a list of all 60 features

  features_list = []

  features_list.append(find_stoich(formula))
  features_list.append(find_elemental_properties(formula))
  features_list.append(find_val_orbs(formula, val_s, val_p, val_d, val_f))
  features_list.append(find_ionic(formula))

  return features_list

# a) Stoichiometric attributes

In [34]:
def find_stoich(formula):

  # Returns a list with P0, P2, and P3 norms

  attributes_list = []

  comp = pmg.Composition(formula)
  num_elms = len(comp.elements)
  attributes_list.append(num_elms) # P0 norm is number of elements

  atom_frac_p2 = 0
  atom_frac_p3 = 0
  denom = 0

  for i in range(num_elms): # First, find denominator value
    num = comp[comp.elements[i-1]]
    denom += num

  for i in range(num_elms): # Then, calculate P2 and P3 norms
    num = comp[comp.elements[i-1]]
    atom_frac_p2 += (num/denom)**2
    atom_frac_p3 += (num/denom)**3

  atom_frac_p2 = atom_frac_p2**(1/2)
  atom_frac_p3 = atom_frac_p3**(1/3)

  attributes_list.append(atom_frac_p2)
  attributes_list.append(atom_frac_p3)

  return attributes_list

# Test
print('Fe2O3: ', find_stoich('Fe2O3'))
print('SrTiO3: ', find_stoich('SrTiO3'))

Fe2O3:  [2, 0.7211102550927979, 0.6542132620377179]
SrTiO3:  [3, 0.66332495807108, 0.6144633651371695]


# b) Elemental-property-based attributes

In [35]:
# Group numbers
column = {'Fe':8, 'O':16, 'Sr':2, 'Ti':4}
row = {'Fe':4, 'O':2, 'Sr':5, 'Ti':6}

# Elemental properties
properties = ['Z', 'atomic_mass', 'atomic_radius', 'X']
custom_properties = {'column':column, 'row':row, 'val_s':val_s, 'val_p':val_p,
                     'val_d':val_d, 'val_f':val_f}


def stats(prop_name, prop_vals, atomic_fractions):

  # Returns statistical values for each target property

  for elm in prop_vals:
    mean = np.sum([atomic_fractions[elm] * prop_vals[elm]])
    dev = np.sum([atomic_fractions[elm] * abs(prop_vals[elm] - mean)])

  min_val = min(prop_vals.values())
  max_val = max(prop_vals.values())
  range = max_val - min_val

  return {prop_name: {'min':min_val, 'max':max_val, 'range':range,
                          'fraction-weighted mean':mean, 'average deviation':dev}}


def find_elemental_properties(formula):

  # Returns a large dictionary with all elemental properties

  comp = pmg.Composition(formula)

  results = {}
  atomic_fractions = {}
  total_atoms = sum(comp.values())
  for elm, count in comp.items():
    atomic_fractions[elm] = count/total_atoms

  # Calculating properties
  for prop in properties:
    prop_vals = {elm.symbol:getattr(elm, prop) for elm in comp.keys()}
    results.update(stats(prop, prop_vals, {el.symbol:af for el, af in atomic_fractions.items()}))

  # Calculating custom properties
  for prop, dictionary in custom_properties.items():
    prop_vals = {elm.symbol:dictionary[elm.symbol] for elm in comp.keys()}
    results.update(stats(prop, prop_vals, {el.symbol: af for el, af in atomic_fractions.items()}))

  return results

# Test
print('Fe2O3: ', find_elemental_properties('Fe2O3'))
print('SrTiO3: ', find_elemental_properties('SrTiO3'))

Fe2O3:  {'Z': {'min': 8, 'max': 26, 'range': 18, 'fraction-weighted mean': 4.8, 'average deviation': 1.92}, 'atomic_mass': {'min': 15.9994, 'max': 55.845, 'range': 39.8456, 'fraction-weighted mean': 9.599639999999999, 'average deviation': 3.839856}, 'atomic_radius': {'min': 0.6, 'max': 1.4, 'range': 0.7999999999999999, 'fraction-weighted mean': 0.36, 'average deviation': 0.144}, 'X': {'min': 1.83, 'max': 3.44, 'range': 1.6099999999999999, 'fraction-weighted mean': 2.064, 'average deviation': 0.8255999999999999}, 'column': {'min': 8, 'max': 16, 'range': 8, 'fraction-weighted mean': 9.6, 'average deviation': 3.84}, 'row': {'min': 2, 'max': 4, 'range': 2, 'fraction-weighted mean': 1.2, 'average deviation': 0.48}, 'val_s': {'min': 2, 'max': 2, 'range': 0, 'fraction-weighted mean': 1.2, 'average deviation': 0.48}, 'val_p': {'min': 0, 'max': 4, 'range': 4, 'fraction-weighted mean': 2.4, 'average deviation': 0.96}, 'val_d': {'min': 0, 'max': 6, 'range': 6, 'fraction-weighted mean': 0.0, 'aver

# c) Valence orbital occupation attributes

In [46]:
def find_val_orbs(formula, val_s, val_p, val_d, val_f):

  # Returns dictionary with fraction-weighted average of the number of
  # valence electrons in each orbital

  comp = pmg.Composition(formula)
  total_val_e = 0
  val = {'s':0, 'p':0, 'd':0, 'f':0}

  atomic_fractions = {}
  total_atoms = sum(comp.values())
  for elm, count in comp.items():
    atomic_fractions[elm] = count/total_atoms

  # Valence electrons and contributions from all orbital types
  for elm, frac in atomic_fractions.items():
    elm_symb = elm.symbol
    total_val = (val_s[elm_symb] + val_p[elm_symb] + val_d[elm_symb] + val_f[elm_symb])
    total_val_e += frac * total_val
    val['s'] += frac * val_s[elm_symb]
    val['p'] += frac * val_p[elm_symb]
    val['d'] += frac * val_d[elm_symb]
    val['f'] += frac * val_f[elm_symb]

  orbital_fractions = {orbital:val[orbital] / total_val_e for orbital in val}

  return orbital_fractions

# Test
print('Fe2O3: ', find_val_orbs('Fe2O3', val_s, val_p, val_d, val_f))
print('SrTiO3: ', find_val_orbs('SrTiO3', val_s, val_p, val_d, val_f))

Fe2O3:  {'s': 0.29411764705882354, 'p': 0.35294117647058826, 'd': 0.3529411764705883, 'f': 0.0}
SrTiO3:  {'s': 0.4166666666666667, 'p': 0.5, 'd': 0.08333333333333334, 'f': 0.0}


# d) Ionic compound attributes

In [56]:
def find_ionic(formula):

  # Returns the possibility of the compound being neutral, max ionic character,
  # and mean ionic character

  comp = pmg.Composition(formula)
  elms = list(comp.elements)
  max_ionic = 0
  sum_ionic = 0
  ionic_pairs = 0

  # A single element will not form an ionic compound
  if len(elms) ==1:
    return False, max_ionic, 0

  # Atomic fractions and oxidation states
  total_atoms = sum(comp.get_el_amt_dict().values())
  atomic_fractions = {elm.symbol:comp.get_el_amt_dict()[elm.symbol] / total_atoms for elm in elms}
  oxidation_states = {elm.symbol:elm.common_oxidation_states for elm in elms}

  for state_comb in itertools.product(*(oxidation_states[el] for el in oxidation_states)):
    if np.isclose(sum(frac * state for frac, state in zip(atomic_fractions.values(), state_comb)), 0 , atol=1e-3):
      possible_neutral = True
      break

  for i in range(len(elms)):
    for j in range(i + 1, len(elms)):
      Xi = elms[i].X
      Xj = elms[j].X
      ionic_char = 1 - np.exp(-(Xi-Xj)**2 / 4)
      max_ionic = max(max_ionic, ionic_char)
      sum_ionic += (atomic_fractions[elms[i].symbol] * atomic_fractions[elms[j].symbol] * ionic_char)
      ionic_pairs += 1


  mean_ionic_char = sum_ionic / ionic_pairs if ionic_pairs > 0 else 0

  return possible_neutral, max_ionic, mean_ionic_char

# Test
print('Fe2O3 ionic features: ', find_ionic('Fe2O3'))
print('SrTiO3 ionic features: ', find_ionic('SrTiO3'))

Fe2O3 ionic features:  (True, 0.47692216400686216, 0.11446131936164691)
SrTiO3 ionic features:  (True, 0.7877573323062756, 0.05639939070315526)


# Test

In [58]:
iron_oxide = find_features('Fe2O3')
strontium_tin_oxide = find_features('SrTiO3')

print('Fe2O3 attributes', '\n')
for i in iron_oxide:
  print(i, '\n')

print('\n', 'SrTiO3 attributes', '\n')
for i in strontium_tin_oxide:
  print(i, '\n')

Fe2O3 attributes 

[2, 0.7211102550927979, 0.6542132620377179] 

{'Z': {'min': 8, 'max': 26, 'range': 18, 'fraction-weighted mean': 4.8, 'average deviation': 1.92}, 'atomic_mass': {'min': 15.9994, 'max': 55.845, 'range': 39.8456, 'fraction-weighted mean': 9.599639999999999, 'average deviation': 3.839856}, 'atomic_radius': {'min': 0.6, 'max': 1.4, 'range': 0.7999999999999999, 'fraction-weighted mean': 0.36, 'average deviation': 0.144}, 'X': {'min': 1.83, 'max': 3.44, 'range': 1.6099999999999999, 'fraction-weighted mean': 2.064, 'average deviation': 0.8255999999999999}, 'column': {'min': 8, 'max': 16, 'range': 8, 'fraction-weighted mean': 9.6, 'average deviation': 3.84}, 'row': {'min': 2, 'max': 4, 'range': 2, 'fraction-weighted mean': 1.2, 'average deviation': 0.48}, 'val_s': {'min': 2, 'max': 2, 'range': 0, 'fraction-weighted mean': 1.2, 'average deviation': 0.48}, 'val_p': {'min': 0, 'max': 4, 'range': 4, 'fraction-weighted mean': 2.4, 'average deviation': 0.96}, 'val_d': {'min': 0, '